<a href="https://colab.research.google.com/github/Danub2002/Leads_scrape/blob/main/Leads_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import requests
import re
import googlemaps
from bs4 import BeautifulSoup

In [ ]:
KEY = "" # Enter your API Key
query = "" # Your query that you want to seach on google maps
business_type = "" # Business type that you are searching for

gmaps = googlemaps.Client(KEY)

In [ ]:
def text_match(text):
  """
  Checks if the address of the bussiness if located where desired
  """

  patterns = '.- BA,|- PE,|State of Bahia.' # Setted to filter business only in Bahia and Pernambuco
  if re.search(patterns,  text):
    return True
  else:
    return False
      

True


In [ ]:
def create_cordinates_grid(desired_grid_lenght,results):
  NORTHEAST_LAT = results[0]["geometry"]["viewport"]["northeast"]["lat"]
  NORTHEAST_LNG = results[0]["geometry"]["viewport"]["northeast"]["lng"]
  SOUTHWEST_LAT = results[0]["geometry"]["viewport"]["southwest"]["lat"]
  SOUTHWEST_LNG = results[0]["geometry"]["viewport"]["southwest"]["lng"]
  
  # Cordinates of the state of Bahia
  # NORTHEAST_LAT = -8.5328205 
  # NORTHEAST_LNG = -37.3411465
  # SOUTHWEST_LAT = -18.3485603  
  # SOUTHWEST_LNG = -46.5772847 
  
  desired_grid_lenght = 10

  
  epsilon = 0.0000001; 
  intermediate_grid_length = desired_grid_lenght - 1

  lat_step_size = ( NORTHEAST_LAT - SOUTHWEST_LAT ) / intermediate_grid_length
  lng_step_size = ( NORTHEAST_LNG - SOUTHWEST_LNG ) / intermediate_grid_length
  lat = SOUTHWEST_LAT 

  cordinate_grid = []

  while lat <= NORTHEAST_LAT + epsilon:
    lng = SOUTHWEST_LNG
    
    while lng <=  NORTHEAST_LNG + epsilon:
      cordinate = [lat,lng]

      cordinate_grid.append(cordinate)
      lng += lng_step_size
      
    lat += lat_step_size
  
  return cordinate_grid 

cordinate_grid = create_cordinates_grid(10)


# cordinate_grid = [[-18.3485603,-46.5772847],
# [-18.3485603,-41.9592156],
# [-18.3485603,-37.3411465],
# [-13.4406904,-46.5772847],
# [-13.4406904,-41.9592156],
# [-13.4406904,-37.3411465],
# [-8.5328205,-46.5772847],
# [-8.5328205,-41.9592156],
# [-8.5328205,-37.3411465]]


In [ ]:



def get_phone_number_and_city(id):
  """
    Gets the details of a bussiness based on it's id
  """
  res = gmaps.place(place_id = id)
  phone_number = res["result"]["formatted_phone_number"]

  html = BeautifulSoup(res["result"]["adr_address"])
  city = html.find("span", attrs = {"class" : "locality"}).text
  

  return phone_number,city

def get_data(cordinate_grid,query,type):
  data = []

  """"
    Looks for 60 bussines for each cordinate of the grid
  """
  for cordinate in cordinate_grid:
    lat = cordinate[0]
    lng = cordinate[1]

    
    for i in range(3):
      next_page_token = ""
      try:

        if i == 0:

          response = gmaps.places(query = query, location=(lat,lng), type = type)
          results = response["results"]
          next_page_token = response["next_page_token"]

        elif i == 2:
          response = gmaps.places(query = query, location=(lat,lng), type = type, page_token = next_page_token)
          results = response["results"]
          
        else:
          response = gmaps.places(query = query, location=(lat,lng), type = type)
          results = response["results"]
          next_page_token = response["next_page_token"]

        for result in results:
          
            if text_match(result["formatted_address"]):
              id = result["place_id"]
              name = result["name"]
              address = result["formatted_address"]

              phone_number,city = get_phone_number_and_city(id)

              place = [id,name,address,city,phone_number]
              data.append(place)
      except Exception as e:
        print(e)

  return data

In [ ]:
data = get_data(cordinate_grid,query,type = business_type)

# The DataFrame contains info about the name, address, city and the phone number of each business
df = pd.DataFrame(data, columns = ["id","name","address","city","phone_number"])


'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'next_page_token'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'next_page_token'
'next_page_token'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'next_page_token'
'next_page_token'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_number'
'formatted_phone_numbe

In [ ]:
# Eliminating the duplicated results

df = df.drop_duplicates()
df

,id,name,address,city,phone_number
0,ChIJoV0TWxBENQcReKTLP6qYZr8,Tiago Chaveiro,"Av. Pres. Getúlio Vargas, 3701 - Centro, Teixe...",Teixeira de Freitas,(73) 99831-4586
1,ChIJzRpSww1ENQcRSXbbTu0J4Hg,Chaveiro Imperial,"Av. Pres. Getúlio Vargas, 4660 - Centro, Teixe...",Teixeira de Freitas,(73) 99972-5173
2,ChIJBRqa_AFFNQcRarW8SoAOKV0,Chaveiro Confiança,"Av. Pres. Getúlio Vargas, 3701 - Centro, Teixe...",Teixeira de Freitas,(73) 99990-8983
3,ChIJ34nlvYpHNQcRzc3uHK3_ZeQ,Bento Chaveiro,"Rua Gonçalves e, Av. Gonçalves Lêdo, 565A - Be...",Teixeira de Freitas,(73) 99964-2192
12,ChIJz5dCKXnDygARPelkDUmF5Zc,Paulista Chaveiro,"Tv. Ilhéus, 24B - Centro, Mucuri - BA, 45930-000",Mucuri,(73) 98205-9331
...,...,...,...,...,...
2160,ChIJKUjHkswbqAcRb71e-NuQjiE,CP Chaves e Carimbos | Chaveiro,"R. Alcídes Cursino, 108 - Centro, Arcoverde - ...",Arcoverde,(87) 99918-6478
2163,ChIJc6nGAHt0qQcRkAlfPXT800I,Chaveiro Multchaves,"R. Joaquim Nabuco, 231A - Santa Tereza, Santa ...",Santa Cruz do Capibaribe,(81) 99361-2623
2189,ChIJRWZiCGOLqQcR1zUIgncA0_U,CHAVEIRO ESPECIALIZADO,Rua São Julião 27 Nossa Senhora das Dores - Ce...,Caruaru,(81) 7105-5051
2190,ChIJ3y0g1HSoqAcR0yqTYqX0ucY,BRUNO CHAVEIRO AUTOMOTIVO,"R. Santa Terezinha, Nº42 - CENTRO, São José do...",São José do Egito,(87) 99935-7877


In [ ]:
# Exporting the data
df.to_excel("leads.xlsx")